In [1]:
import pandas as pd

donations_df = pd.read_excel('./data/ForeignGifts.xls', skiprows=1)
# donations_df.loc[:, 'Foreign Gift Amount'] = donations_df.loc[:, 'Foreign Gift Amount'].apply('{:,}'.format)
donations_df.head()

,ID,OPEID,Institution Name,City,State,Foreign Gift Received Date,Foreign Gift Amount,Gift Type,Country of Giftor,Giftor Name
0,1,324200,Carnegie Mellon University,Pittsburgh,PA,2015-01-01,77391377,Contract,QATAR,Qatar Foundation for Education
1,2,143400,American University (The),Washington,DC,2012-01-25,500000,Monetary Gift,BAHRAIN,Kingdom of Bahrain
2,3,143400,American University (The),Washington,DC,2013-05-28,500000,Monetary Gift,BAHRAIN,Sheikh Salman Bin Al-Khalifa
3,4,143400,American University (The),Washington,DC,2015-09-10,500000,Monetary Gift,KUWAIT,NaN
4,5,143400,American University (The),Washington,DC,2012-05-17,610000,Monetary Gift,ENGLAND,NaN


In [2]:
donations_df.groupby('Institution Name').count()['Foreign Gift Amount'].sort_values(ascending=False)

schools = donations_df['Institution Name'].unique()
school_lst = []
for school in schools:
    school_lst.append(school.split(' (')[0])

In [ ]:
import time
import numpy as np
from geopy.geocoders import Nominatim
from geopy import exc


geolocator = Nominatim(user_agent='foerign_donations')
schools_df = pd.DataFrame(columns=['Latitude', 'Longitude'])
school_names_to_fix = []
for school in school_lst:
    try:
        location = geolocator.geocode(school)
        if location.address.split(' ')[-1].strip() != 'America':
            school_names_to_fix.append(school)
            print(school + '\t' + 'SKIPPED')
            continue
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude
        print(school + '\t' + location.address.split(' ')[-1])
    except AttributeError:
        schools_df.at[school, 'Latitude'] = np.nan
        schools_df.at[school, 'Longitude'] = np.nan
        school_names_to_fix.append(school)
        print(school + '\t' + 'SKIPPED')
    except exc.GeocoderTimedOut:
        print('Paused... Waiting for server.')
        time.sleep(10)
        location = geolocator.geocode(school)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude        
    time.sleep(1)

Carnegie Mellon University	America
American University	America
Amherst College	America
Arizona State University	America
Michigan State University	America
University of Colorado Boulder	America
Massachusetts Institute of Technology	America
Virginia Commonwealth University	America
California Institute of Technology	America
University of Arkansas	America
Georgetown University	America


KeyboardInterrupt: 

In [ ]:
school_names_to_fix

In [ ]:
new_school_names = {'Johns Hopkis University': 'Johns Hopkins University, USA',
                    'New York Institute of Technology': 'New York Institute of Technology, USA',
                    'Teachers College, Columbia University': 'Teachers College',
                    'California Institute of Advanced Management': 'Platt College Los Angeles',
                    'Indiana University - Purdue University Indianapolis': 'IUPUI',
                    'University of North Texas Health Science Center at Fort Worth': 'UNTHSC',
                    'Rochester Institute of Technology': 'RIT, USA',
                    'University of Vermont and State Agricultural College': 'University of Vermont',
                    'Iowa State University of Science & Technology': 'Iowa State University',
                    'Polytechnic Institute of New York University': 'New York City College of Technology',
                    'Keck Graduate Institute of Applied Life Sciences': 'Keck Graduate Institute',
                    'St. Lawrence University': 'Saint Lawrence University, USA',
                    'Valencia College': 'Valencia College West Campus',
                    'Western International University': 'Fountainhead Pkwy, Arizona'
                   }

for school, hack_name in new_school_names.items():
    try:
        location = geolocator.geocode(hack_name)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude
        print(school + '\t' + hack_name + '\t' + location.address + '\n')
    except AttributeError:
        schools_df.at[school, 'Latitude'] = np.nan
        schools_df.at[school, 'Longitude'] = np.nan
        school_names_to_fix.append(school)
    except exc.GeocoderTimedOut:
        time.sleep(10)
        schools_df.at[school, 'Latitude'] = location.latitude
        schools_df.at[school, 'Longitude'] = location.longitude        
    time.sleep(1)

In [ ]:
schools_df.isnull().values.any()

In [ ]:
school_names_map = {}
for split_name, orig_name in zip(schools_df.index, schools):
    school_names_map[split_name] = orig_name

schools_df_final = schools_df.rename(school_names_map)

In [ ]:
merged_df = pd.read_csv('donation_democracy_index.csv')

In [ ]:
country_lst = list(merged_df.groupby('Country of Giftor').count().index)
country_df = pd.DataFrame(index=country_lst, columns=['Latitude', 'Longitude'])
country_name_to_fix = []
for country in country_lst:
    try:
        location = geolocator.geocode(country)
        country_df.at[country, 'Latitude'] = location.latitude
        country_df.at[country, 'Longitude'] = location.longitude
    except AttributeError:
        country_df.at[country, 'Latitude'] = np.nan
        country_df.at[country, 'Longitude'] = np.nan
        country_name_to_fix.append(country)
    except exc.GeocoderTimedOut:
        time.sleep(10)
        country_df.at[country, 'Latitude'] = location.latitude
        country_df.at[country, 'Longitude'] = location.longitude
    print(country, location.latitude, location.longitude)
    time.sleep(1)  

In [ ]:
merged_country_df = pd.merge(merged_df, country_df, left_on='Country of Giftor', right_index=True, how='outer')
merged_country_df = merged_country_df.rename({'Latitude': 'Country Latitude', 'Longitude': 'Country Longitude'}, axis=1)

In [ ]:
merged_final_df = pd.merge(merged_country_df, schools_df_final, left_on='Institution Name', right_index=True, how='outer')
merged_final_df = merged_final_df.rename({'Latitude': 'School Latitude', 'Longitude': 'School Longitude'}, axis=1)
merged_final_df.to_csv('merged_data_w_coord.csv')

In [ ]:
merged_final_df.columns